In [81]:
# Importa as bibliotecas necessárias
import gensim
import gensim.corpora as corpora
import pandas as pd
import pyLDAvis.gensim
import re
import spacy

from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
from gensim.models import LdaMulticore

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

from tqdm import tqdm
tqdm.pandas()

## Carregamento dos Dados e Análise Exploratória

In [14]:
# Carrega o dataset referente às reviews do Airbnb Rio de Janeiro
reviews = pd.read_csv("data/reviews.csv")

In [15]:
# Apresenta as primeiras 5 reviews
reviews.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r\n...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


In [4]:
# Verifica os tipos de variáveis
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292516 entries, 0 to 292515
Data columns (total 7 columns):
listing_id       292516 non-null int64
id               292516 non-null int64
date             292516 non-null object
reviewer_id      292516 non-null int64
reviewer_name    292516 non-null object
comments         292446 non-null object
language         292076 non-null object
dtypes: int64(3), object(4)
memory usage: 15.6+ MB


In [5]:
# Converte a coluna comments para string
reviews["comments"] = reviews["comments"].astype(str)

In [122]:
# Adiciona coluna com idioma da review
def try_detect_language(text):
    """
    Dado uma string text, tenta obter o idioma predominante.
    
    Argumentos:
    text -- (str) o texto para o qual se deseja detectar o idioma
    Retorna: (str) o código do idioma (e.g. 'en' para inglês). 
             Retorna uma string vazia caso não seja possível detectar o idioma
    """
    try:
        return detect(text)
    except:
        return ""

reviews["language"] = reviews["comments"].apply(try_detect_language)

In [123]:
# Verifica total de reviews para cada idioma identificado e apresenta os 5 idiomas mais comuns
reviews.language.value_counts().head()

pt    138315
en     83736
es     54358
fr      8424
de      2057
Name: language, dtype: int64

## Preprocessamento dos Dados

In [6]:
nlp_pt = spacy.load("pt")

def preprocess(text, 
               min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE'],
               nlp = nlp_pt): 
    """
    Dado text, min_token_len, e irrelevant_pos, faz o preprocessamento do texto
    
    Argumentos:
    text -- (str) texto a ser pré-processado
    min_token_len -- (int) o menor comprimento de token a ser considerado
    irrelevant_pos -- (list) lista as classes gramaticais a serem ignoradas
    
    Retorna: (str) texto pré processado
    """
    
    # Converte todo os texto para apenas caractéres minúsculos
    text = text.lower()
    
    # Remove termos utilizados pelo airbnb para ocultar informações
    text = re.sub(r'\((.*hidden by airbnb)\)', "", text)
    
    # Processa o texto usando spacy
    doc = nlp(text)
    
    preprocessed = []
    
    # Remove stop words, pontuação, tokens menores que min_token_len e tokens cuja 
    # classe gramatical identificada é irrelavante para análise
    for token in doc:
        if not token.is_stop and not token.is_punct and len(token) >= min_token_len and token.pos_ not in irrelevant_pos:
            # Armazena o lemma do token na variável de saída
            preprocessed.append(token.lemma_)
            
    return " ".join(preprocessed)

def preprocess_pt(text):
    return preprocess(text, nlp = nlp_pt)

In [7]:
# Filtra as reviews escritas em Português
reviews_pt = reviews[reviews.language == "pt"]

In [8]:
# Aplica a função de preprocessamento a todas as reviews e armazena o resultado numa nova coluna
reviews_pt["clean_comments"] = reviews_pt.comments.progress_apply(preprocess_pt)

100%|██████████| 138315/138315 [1:07:43<00:00, 34.04it/s]    
/Users/marcelle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
# Salva um .csv com as reviews preprocessadas
reviews_pt.to_csv('data/reviews_pt_preprocessed.csv', index = False)

In [46]:
preprocessed_corpus = [doc.split() for doc in reviews_pt.clean_comments.tolist()]

# Cria um vocabulário para o modelo LDA e converte
# o corpus em uma matriz documento-termo para LDA
dictionary = corpora.Dictionary(preprocessed_corpus)

# Remove palavras que aparecem em menos de no_below reviews e em mais do que no_above % das reviews
dictionary.filter_extremes(no_below = 10, no_above = 0.1, keep_n = 10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in preprocessed_corpus]

In [124]:
# Verifica o número total de termos no dicionário
len(dictionary)

5888

## Escolha de Hiperparâmetro e Criação do Modelo

In [48]:
# Treina diferentes modelos LDA, com num_topics variando de 5 a 13
num_topics = range(5, 14)
passes = 10

lda_models = {}

for n in num_topics:
    print('Number of topics = ', n)
    
    lda = LdaMulticore(corpus = doc_term_matrix,
                       id2word = dictionary,
                       num_topics = n,
                       workers = 3,
                       passes = passes)
    
    lda_models[str(n)] = lda

    cm = CoherenceModel(model = lda, 
                        texts = preprocessed_corpus,
                        dictionary = dictionary,
                        coherence = 'c_v')
    
    print(lda.print_topics(), '\n')
    print("Coherence:", cm.get_coherence())

Number of topics =  5
[(0, '0.022*"aptar" + 0.022*"foto" + 0.017*"organizar" + 0.016*"prestativo" + 0.015*"responder" + 0.015*"aconchegante" + 0.013*"rápido" + 0.012*"condomínio" + 0.012*"adorar" + 0.010*"indicar"'), (1, '0.025*"cama" + 0.018*"cozinhar" + 0.014*"banheiro" + 0.013*"problema" + 0.013*"ar" + 0.012*"condicionar" + 0.011*"roupar" + 0.010*"haver" + 0.010*"ter" + 0.009*"dia"'), (2, '0.062*"check" + 0.048*"in" + 0.041*"horário" + 0.035*"flexível" + 0.030*"check-in" + 0.027*"out" + 0.020*"preço" + 0.019*"host" + 0.018*"check-out" + 0.017*"combinar"'), (3, '0.023*"maravilhoso" + 0.017*"incrível" + 0.016*"sentir" + 0.015*"visto" + 0.013*"pessoa" + 0.013*"deixar" + 0.012*"receber" + 0.012*"experiência" + 0.011*"aconchegante" + 0.011*"vontade"'), (4, '0.030*"metrô" + 0.021*"bar" + 0.020*"supermercado" + 0.019*"mercar" + 0.019*"fácil" + 0.017*"copacabana" + 0.017*"acesso" + 0.016*"farmácia" + 0.013*"tranquilo" + 0.012*"padaria"')] 

Coherence: 0.5618693571383846
Number of topics =  

Coherence: 0.5873607644226186
Number of topics =  11
[(0, '0.020*"dia" + 0.016*"cidade" + 0.013*"noite" + 0.013*"chegar" + 0.012*"bairro" + 0.012*"tranquilo" + 0.011*"acesso" + 0.011*"passar" + 0.011*"lapa" + 0.010*"centrar"'), (1, '0.039*"deixar" + 0.038*"dica" + 0.035*"dar" + 0.030*"ajudar" + 0.029*"vontade" + 0.022*"dúvida" + 0.020*"receber" + 0.018*"precisar" + 0.015*"chegar" + 0.014*"café"'), (2, '0.060*"foto" + 0.028*"prestativo" + 0.027*"fácil" + 0.026*"organizar" + 0.022*"simpático" + 0.020*"agradável" + 0.020*"tranquilo" + 0.020*"aptar" + 0.020*"acesso" + 0.016*"educar"'), (3, '0.062*"metrô" + 0.041*"supermercado" + 0.040*"mercar" + 0.039*"bar" + 0.033*"farmácia" + 0.025*"padaria" + 0.021*"quadrar" + 0.021*"estação" + 0.020*"copacabana" + 0.018*"comércio"'), (4, '0.050*"visto" + 0.042*"condomínio" + 0.024*"piscina" + 0.021*"estruturar" + 0.019*"barrir" + 0.018*"flat" + 0.018*"preço" + 0.016*"lindo" + 0.015*"frente" + 0.014*"prédio"'), (5, '0.047*"check" + 0.044*"rápido" + 0.03

Analisando o Coherence score, o melhor modelo seria aquele treinado com `num_topics = 12`. Entretanto, analisando a interpretabilidade dos tópicos identificados, esse modelo deixa a desejar, uma vez que há uma sobreposição de temas em diferentes tópicos. Assim, é preferível selecionar um modelo com menor número de tópicos, que vai oferecer resultados mais interpretáveis

In [61]:
# Seleciona o modelo treinado com num_topics = 5
lda_model = lda_models["5"]

# Salva o modelo
lda_model.save("model/lda")

## Visualização dos Tópicos

In [66]:
# Gera uma visualização dos tópicos e palavras associadas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
vis

/Users/marcelle/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.132323  0.107377       1        1  19.359997
1      0.047069 -0.279948       2        1  23.678696
2     -0.399279  0.039971       3        1   5.785853
3      0.130524  0.074789       4        1  28.288708
4      0.089364  0.057810       5        1  22.886744, topic_info=     Category          Freq          Term         Total  loglift  logprob
271   Default   6248.000000         check   6248.000000  30.0000  30.0000
100   Default  12026.000000         metrô  12026.000000  29.0000  29.0000
270   Default  10260.000000          cama  10260.000000  28.0000  28.0000
61    Default   4786.000000            in   4786.000000  27.0000  27.0000
269   Default   8214.000000           bar   8214.000000  26.0000  26.0000
147   Default   7967.000000  supermercado   7967.000000  25.0000  25.0000
273   Default   4061.000000       horário   4061.000000  24.0000  24.0000
216   Default   7679.000000        mercar   7679.000000  23.0000  23.0000
90    Default   7203.000000      cozinhar   7203.000000  22.0000  22.0000
470   Default   3525.000000      flexível   3525.000000  21.0000  21.0000
263   Default   7840.000000         fácil   7840.000000  20.0000  20.0000
197   Default   6362.000000      farmácia   6362.000000  19.0000  19.0000
505   Default   6769.000000        acesso   6769.000000  18.0000  18.0000
83    Default  10017.000000         aptar  10017.000000  17.0000  17.0000
338   Default   3118.000000      check-in   3118.000000  16.0000  16.0000
371   Default   8593.000000      incrível   8593.000000  15.0000  15.0000
4     Default  14191.000000   maravilhoso  14191.000000  14.0000  14.0000
166   Default   5661.000000      banheiro   5661.000000  13.0000  13.0000
149   Default   7245.000000    copacabana   7245.000000  12.0000  12.0000
278   Default   2716.000000           out   2716.000000  11.0000  11.0000
85    Default   5272.000000            ar   5272.000000  10.0000  10.0000
56    Default  11701.000000          foto  11701.000000   9.0000   9.0000
200   Default   4881.000000       padaria   4881.000000   8.0000   8.0000
243   Default   3217.000000         preço   3217.000000   7.0000   7.0000
88    Default   4725.000000   condicionar   4725.000000   6.0000   6.0000
185   Default   4672.000000        roupar   4672.000000   5.0000   5.0000
146   Default   9342.000000        sentir   9342.000000   4.0000   4.0000
126   Default   4607.000000      comércio   4607.000000   3.0000   3.0000
312   Default   5580.000000       vontade   5580.000000   2.0000   2.0000
7     Default   6225.000000      problema   6225.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
258    Topic5   2269.991943     turístico   2277.183594   1.4714  -5.1609
4085   Topic5    276.585266         museu    277.529572   1.4712  -7.2659
126    Topic5   4590.264648      comércio   4607.870117   1.4708  -4.4567
505    Topic5   6626.695312        acesso   6769.584961   1.4533  -4.0895
375    Topic5   1251.017212           sul   1259.323242   1.4680  -5.7567
695    Topic5    513.992554         orlar    515.812744   1.4711  -6.6462
112    Topic5    691.724426          táxi    697.733704   1.4660  -6.3492
122    Topic5   4446.228027         metro   4580.698730   1.4448  -4.4886
217    Topic5   2867.466797       próximo   2967.750732   1.4402  -4.9272
376    Topic5   1342.210571         zonar   1371.541016   1.4530  -5.6863
263    Topic5   7391.535645         fácil   7840.154297   1.4157  -3.9803
15     Topic5   4356.894531       quadrar   4621.615234   1.4156  -4.5089
644    Topic5   1643.954102      caminhar   1695.089844   1.4440  -5.4835
1928   Topic5    859.337830           brt    872.808899   1.4591  -6.1322
149    Topic5   6688.569824    copacabana   7245.311523   1.3947  -4.0802
320    Topic5   3781.915527       ipanema   4036.250000   1.4095  -4.6504
1311   Topic5   2285.588867

In [109]:
# Assinala rótulos para cada um dos tópicos identificados, com base nas palavras que os compõem
topic_labels = {0:'Atendimento',
                1:'Instalações',
                2:'Check-in/Check-out',
                3:'Experiência',
                4:'Localização/Conveniência'}

In [113]:
def get_topic_label_prob(doc_term_row, topic_labels = topic_labels, model = lda_model):
    """
    Dado um vetor da doc_term_matrix, e um modelo LDA já treinado, esta função
    identifica o tópico predominante mais provável e retorna uma string com
    o nome do tópico identificado, conforme assinalado em topic_labels, e a probabilidade
    associada
    
    Argumentos:
    doc_term_row -- (list) o vetor do documento cujo tópico deseja-se identificar
    topic_labels -- (dictionary) dicionário contendo o mapeamento de topic id para topic_label
    model -- (gensim ldamodel) um modelo LDA treinado
    
    Retorna: (str) string no formato
             topic_label:probabilidade
    
    """    

    topic_assignment = sorted(model[doc_term_row], key = lambda x:x[1], reverse = True)
    topic, prob = topic_assignment[0]
    
    return str(topic_labels[topic]) + ":" + str(prob)

In [114]:
topics = []
for row in doc_term_matrix:
    topics.append(get_topic_label_prob(row))

In [120]:
# Adiciona coluna main_topic com o principal tópico identificado pelo modelo para cada review, e a probabilidade correspondente
reviews_pt["main_topic"] = topics
reviews_pt.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,language,clean_comments,main_topic
18,17878,597736,2011-10-04,989464,Jose Mauro,Muito Melhor do que ir para um hotel. Apt com ...,pt,hotel apt excelente localização quarteirão pra...,Atendimento:0.44995096
19,17878,620065,2011-10-11,1157847,Gabriel,"Muito bom o apartamento, a pouco mais de uma q...",pt,apartamento quadrar praia lugar comer sair,Localização/Conveniência:0.83668894
33,17878,1566424,2012-06-26,2253138,Sergio,"o apartamento é muito confortável, e todo equi...",pt,apartamento confortável equipar unica reivindi...,Instalações:0.93271095
49,17878,3451132,2013-02-01,4142117,Laura,O apartamento é muito bom. Quartos e sala gran...,pt,apartamento quarto sala decoração gostar areja...,Instalações:0.60886776
70,17878,5100450,2013-06-12,5538405,Jose Bertoldo,"Bom apartamento, limpo, boa localização, porta...",pt,apartamento limpar localização portar eficient...,Atendimento:0.60195154


In [125]:
# Salva .csv das reviews incluindo a coluna main_topic
reviews_pt.to_csv('data/reviews_pt_with_topics.csv', index = False)

## Conclusões

Uma vez que não existem categorias pré-definidas para as reviews, não é possível medir a precisão do modelo desenvolvido. Entretanto, de forma subjetiva, é possível verificar que os tópicos identificados, tem relação com o conteúdo das reviews. Uma abordagem que permitiria obter uma métrica mais concreta para avaliação do modelo, ao custo de um maior tempo de desenvolvimento, seria usar uma parte das reviews para fazer o treinamento do modelo, e a outra parte seria rotulada manualmente para posteriormente ser utilizada como set de validação, comparando os rótulos manuais com os sugeridos pelo modelo.

#### Análises Futuras

A análise em questão foi limitada ao idioma português, e no futuro, seria interessante incluir as reviews de outros idiomas. Uma opção para tal seria efetuar o preprocessamento utilizando os modelos correspondentes do spaCy para cada um dos idiomas, e posteriormente utilizando [tradução automática](https://pypi.org/project/googletrans/) nas reviews preprocessadas para realizar a segmentação por tópico.

Ainda com relação ao preprocessamento, seria interessante avaliar mais a fundo formas de refiná-lo ainda mais, identificando termos que poderiam ser desconsiderados, de forma similar ao tratamento dado para as strings que substituem conteúdo ocultado pelo Airbnb (e.g. "website hidden by airbnb").

Outro ponto a ser avaliado seria o impacto de incluir bigrams e trigrams na modelagem, avaliando se os tópicos identificados podem se tornar mais relevantes e interpretáveis, e com menos sobreposição de termos.

Finalmente, para efeitos de validação, vale reforçar o ponto mencionado anteriormente relativo a separação dos dados utilizados para treinamento e validação. Para tal, seria necessário um esforço significativo para rotulação dos dados a serem utilizados para validação, mas com um impacto significativo no que se refere a obter uma avaliação do modelo com mais confiança.